<a href="https://colab.research.google.com/github/Talha1818/Machine-Learning-Python-Notebook-TransData/blob/master/LLM_Model_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.4 MB/s eta 0:00:00


In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs')
model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/Annotation Folder/LLM/gpt2_decoder_classification_model_100_2_epochs')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)

In [34]:
def prediction_update(text, max_length=300, temperature=0.1, top_k=0, top_p=0.9):
    prompt = text
    prompt_inp = "<startofstring> " + prompt + " <bot>:"

    model.config.pad_token_id = model.config.eos_token_id

    encoding = tokenizer.encode_plus(prompt_inp, padding='max_length', truncation=True, max_length=256, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p
    )

    # Decode the generated tokens back into text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Find the index of "<bot>:" and extract the text after it
    index = generated_text.find("<bot>:")
    extracted_text = generated_text[index + len("<bot>:"):].strip()

    return extracted_text


In [7]:
import pandas as pd
import numpy as np
import string
import re

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

df = pd.read_csv("/content/drive/MyDrive/Annotation Folder/Final_Text_Dataset_22_May_2023.csv")
df['token'] = df['token'].astype(int)
df = df[df['clean_text'].notna()]
df = df[df['token'].notna()]
df1 = df['token'].value_counts().to_frame().reset_index().rename(columns={'index':'token','token':'count'})
df_100 = df1[df1['count']>=100]['token'].tolist()
df_new = df[df['token'].isin(df_100)]
df = df_new[['clean_text', 'token']].copy()

df_token = pd.read_excel("/content/drive/MyDrive/Annotation Folder/field_tokens.xlsx", sheet_name='Lender_field_tokens')
df_token = df_token.rename(columns={'Token.1': 'Meaning', 'Token':'token'})
df_token = df_token[['token', 'Meaning']]

# Merge the dataframes based on the 'Token' column
merged_df = pd.merge(df, df_token, on='token')
merged_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,clean_text,token,Meaning
0,haddonfield city abstract lbs zassokaeib name ...,15,LENDER’S INSTRUCTIONS
1,anorire redseiy srarit bsc mea laa wit bth ipe...,15,LENDER’S INSTRUCTIONS
2,instructions escrowtitleclosing agent uuuuu lo...,15,LENDER’S INSTRUCTIONS
3,loan alta policy endorsements issued within da...,15,LENDER’S INSTRUCTIONS
4,loan signatures borrowers must sign documents ...,15,LENDER’S INSTRUCTIONS


In [10]:
df = merged_df.copy()

In [11]:
def remove_punctuation_and_lower(text):
    # Define punctuation excluding comma
    punctuation = string.punctuation.replace(',', '')
    
    # Remove punctuation excluding comma
    translator = str.maketrans('', '', punctuation)
    text_without_punct = text.translate(translator)
    
    # Remove apostrophe
    text_without_punct = re.sub(r"[’']", '', text_without_punct)
    
    # Convert to lowercase
    text_lower = text_without_punct.lower()
    
    return text_lower

In [54]:
df['Meaning'].value_counts()[:10]

lenders instructions                   6874
buyers cd                              5954
lender mortgage                        5172
lender note                            4798
lender amortizationpayment schedule    4716
deed copy                              4316
lenders borrowers w9                   4021
loan application                       3046
seller cd                              2860
lenders 4506                           2408
Name: Meaning, dtype: int64

In [65]:
df['Meaning'] = df['Meaning'].apply(remove_punctuation_and_lower)
df.sample(5)

,clean_text,token,Meaning
5318,doc umentupload process nfm requires upload ex...,15,lenders instructions
71344,loan rev american neighborhood mortgage accept...,112,lender facts
29391,crosscountry mortgageu xkxattention moving ple...,28,lender note
73212,notice furnishing negative information borrowe...,46,lender negative info
73024,false misleading inaccurate statements borrowe...,94,lender occupancy and financial status affidavit


In [45]:
df['clean_text'][8815]

'city abstract llc alta sellers settlement statement north haddon avenue suite haddonfield file cahd property monroe avenue settlement date prepared bellmawr disbursement date closer linda landis buyer sharee maxwell jordan powell seller vicki hewitt lender lenders corp escription ser primary charges credits sales price property rrtuuucussuccudududcidcy seller credit sasoosscscsscscs prorationsadjustments citytown taxes county sewer rrus sewer payoffspayments payoff cenlar siaseai cscs principal late charge government recording transfer charges transfer tax state deed taxes camden county recording office commissions sssoscscscocscscocscscsdsc sos listing agent commission realty mark advantage selling agent commission keller williams washington township title charges title notary fee city abstract llc title settlement fee city abstract llc miscellaneous charges county sewer ccmua deed prep fee kevin lau sewer borough bellmawr poc seller ttc sis due seller sige produced city abstract llc

In [ ]:
# 8815, 22673, 85254, 51082, 62287, 71344, 73212
# 28979 , 36528

In [67]:
prediction_update(df['clean_text'][73212])

'label: lender negative info'